# 📈 AAPL Stock Analysis & Comparison with S&P 500

**Description:** This notebook fetches historical stock data for selected tickers, enriches with company info, computes metrics (returns, moving averages, volatility), compares with the S&P 500 index, and produces interactive visualizations and CSV outputs ready for dashboards.

In [ ]:
# Install required packages (uncomment if needed in Colab)
# !pip install yfinance plotly --quiet

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

sns.set(style='whitegrid')

print('Imports loaded')

Imports loaded


In [ ]:
# -------------------------------
# Dynamic input for stock tickers
# -------------------------------
tickers_input = input("Enter stock symbols separated by commas (e.g., AAPL,MSFT,TSLA) [default AAPL]: ")
tickers = [t.strip().upper() for t in (tickers_input or 'AAPL').split(',') if t.strip()]
print('Tickers selected:', tickers)


Enter stock symbols separated by commas (e.g., AAPL,MSFT,TSLA) [default AAPL]: AAPL
Tickers selected: ['AAPL']


In [ ]:
# -------------------------------
# Fetch historical stock prices and compute daily returns (robust per-ticker)
# -------------------------------
def fetch_stock_data(tickers, period='30d', interval='1d'):
    all_df = None
    for ticker in tickers:
        df = yf.download(ticker, period=period, interval=interval, progress=False)
        if df.empty:
            print(f'⚠️ Warning: No data fetched for {ticker}')
            continue
        df = df.reset_index()
        # Prefix columns to avoid collisions
        df = df.rename(columns={
            'Open': f'{ticker}_Open', 'High': f'{ticker}_High', 'Low': f'{ticker}_Low',
            'Close': f'{ticker}_Close', 'Adj Close': f'{ticker}_Adj_Close', 'Volume': f'{ticker}_Volume'
        })
        # Daily return
        df[f'{ticker}_Daily_Return'] = df[f'{ticker}_Close'].pct_change() * 100
        if all_df is None:
            all_df = df
        else:
            # merge on Date
            all_df = pd.merge(all_df, df, on='Date', how='outer')
    if all_df is None:
        raise ValueError('No stock data fetched for provided tickers')
    all_df = all_df.sort_values('Date').reset_index(drop=True)
    return all_df

# Fetch
stock_data = fetch_stock_data(tickers)
print('Stock data fetched. Columns:', stock_data.columns.tolist())


Stock data fetched. Columns: [('Date', ''), ('AAPL_Close', 'AAPL'), ('AAPL_High', 'AAPL'), ('AAPL_Low', 'AAPL'), ('AAPL_Open', 'AAPL'), ('AAPL_Volume', 'AAPL'), ('AAPL_Daily_Return', '')]


In [ ]:
# -------------------------------
# Fetch company info (safe)
# -------------------------------
def fetch_company_info(tickers):
    info_list = []
    for ticker in tickers:
        try:
            company = yf.Ticker(ticker)
            info = company.info
        except Exception as e:
            info = {}
        info_list.append({
            'Ticker': ticker,
            'Name': info.get('shortName', 'N/A'),
            'Sector': info.get('sector', 'N/A'),
            'Market_Cap': info.get('marketCap', 0),
            'PE_Ratio': info.get('trailingPE', None)
        })
    return pd.DataFrame(info_list)

company_info = fetch_company_info(tickers)
print('Company info fetched:')
display(company_info)


Company info fetched:


,Ticker,Name,Sector,Market_Cap,PE_Ratio
0,AAPL,Apple Inc.,Technology,3806204002304,38.860004


In [ ]:
# -------------------------------
# Data cleaning & Merge company info
# -------------------------------
merged_data = stock_data.copy()

# Merge company info for each ticker safely
for ticker in tickers:
    info_rows = company_info[company_info['Ticker'] == ticker]
    if info_rows.empty:
        merged_data[f'{ticker}_Sector'] = 'N/A'
        merged_data[f'{ticker}_Market_Cap'] = 0
    else:
        info = info_rows.iloc[0]
        merged_data[f'{ticker}_Sector'] = info['Sector']
        merged_data[f'{ticker}_Market_Cap'] = info['Market_Cap']

# Flatten any MultiIndex columns (safety)
merged_data.columns = [('_'.join([str(c) for c in col if c]) if isinstance(col, tuple) else str(col)).strip('_')
                       for col in merged_data.columns]

# Ensure Date exists
if 'Date' not in merged_data.columns:
    merged_data = merged_data.reset_index()

print('Merged data columns:', merged_data.columns.tolist())


Merged data columns: ['Date', 'AAPL_Close_AAPL', 'AAPL_High_AAPL', 'AAPL_Low_AAPL', 'AAPL_Open_AAPL', 'AAPL_Volume_AAPL', 'AAPL_Daily_Return', 'AAPL_Sector', 'AAPL_Market_Cap']


In [ ]:
# -------------------------------
# Calculate metrics (moving averages, volatility, cumulative returns)
# -------------------------------
for ticker in tickers:
    # find close_col and daily_return_col
    close_col = next((c for c in merged_data.columns if f'{ticker}_Close' in c), None)
    daily_col = next((c for c in merged_data.columns if f'{ticker}_Daily_Return' in c), None)
    if close_col is None or daily_col is None:
        print(f'⚠️ Skipping metrics for {ticker} (missing columns)')
        continue
    merged_data[f'{ticker}_MA_10'] = merged_data[close_col].rolling(window=10).mean()
    merged_data[f'{ticker}_MA_20'] = merged_data[close_col].rolling(window=20).mean()
    merged_data[f'{ticker}_Volatility_10'] = merged_data[daily_col].rolling(window=10).std()
    merged_data[f'{ticker}_Cumulative_Return'] = (1 + merged_data[daily_col]/100).cumprod() - 1

# Also add MA_7 and MA_30 for plotting convenience (using the first ticker's close)
first_ticker = tickers[0]
close_col_first = next((c for c in merged_data.columns if f'{first_ticker}_Close' in c), None)
if close_col_first is not None:
    merged_data['MA_7'] = merged_data[close_col_first].rolling(7).mean()
    merged_data['MA_30'] = merged_data[close_col_first].rolling(30).mean()
    merged_data['Daily_Return'] = merged_data[close_col_first].pct_change() * 100

print('Metrics calculated. Columns:', [c for c in merged_data.columns if any(k in c for k in ['MA_','Volatility','Cumulative','Daily_Return'])][:20])


Metrics calculated. Columns: ['AAPL_Daily_Return', 'AAPL_MA_10', 'AAPL_MA_20', 'AAPL_Volatility_10', 'AAPL_Cumulative_Return', 'MA_7', 'MA_30', 'Daily_Return']


In [ ]:
# -------------------------------
# Save dashboard-ready CSV
# -------------------------------
merged_data.to_csv('dashboard_ready_stock_data.csv', index=False)
print('Saved dashboard_ready_stock_data.csv')


Saved dashboard_ready_stock_data.csv


In [ ]:
# -------------------------------
# Interactive Visualizations (Plotly) - safe for NaNs
# -------------------------------
ticker = tickers[0]
close_col = next((c for c in merged_data.columns if f'{ticker}_Close' in c), None)

# 1. Closing Price + MAs
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=merged_data['Date'], y=merged_data[close_col], mode='lines', name=f'{ticker} Close'))
fig1.add_trace(go.Scatter(x=merged_data['Date'], y=merged_data.get('MA_7'), mode='lines', name='MA 7'))
fig1.add_trace(go.Scatter(x=merged_data['Date'], y=merged_data.get('MA_30'), mode='lines', name='MA 30'))
fig1.update_layout(title=f'{ticker} Closing Price with Moving Averages', xaxis_title='Date', yaxis_title='Price')
fig1.show()

# 2. Daily Returns
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=merged_data['Date'], y=merged_data.get('Daily_Return'), mode='lines', name=f'{ticker} Daily Return'))
fig2.update_layout(title=f'{ticker} Daily Returns (%)', xaxis_title='Date', yaxis_title='Daily Return (%)')
fig2.show()

# 3. Histogram of Daily Returns
fig_hist = go.Figure()
fig_hist.add_trace(go.Histogram(x=merged_data.get('Daily_Return').dropna(), nbinsx=30))
fig_hist.update_layout(title='Distribution of Daily Returns')
fig_hist.show()


In [ ]:
# -------------------------------
# Load S&P 500 data & merge (robust)
# -------------------------------
sp500 = yf.download('^GSPC', start=merged_data['Date'].min(), end=merged_data['Date'].max(), progress=False)
sp500.reset_index(inplace=True)
# Flatten MultiIndex if present
sp500.columns = [('_'.join([str(c) for c in col if c]) if isinstance(col, tuple) else str(col)).strip('_') for col in sp500.columns]
# find close column
close_col_sp = next((c for c in sp500.columns if 'Close' in c), None)
if close_col_sp is None:
    raise ValueError('Could not detect Close column in sp500 data')
sp500 = sp500[['Date', close_col_sp]]
sp500.rename(columns={close_col_sp: 'SP500_Close'}, inplace=True)
sp500['SP500_Daily_Return'] = sp500['SP500_Close'].pct_change() * 100

# Merge
comparison_df = pd.merge(merged_data, sp500, on='Date', how='left')
print('comparison_df ready. Columns:', comparison_df.columns.tolist())


comparison_df ready. Columns: ['Date', 'AAPL_Close_AAPL', 'AAPL_High_AAPL', 'AAPL_Low_AAPL', 'AAPL_Open_AAPL', 'AAPL_Volume_AAPL', 'AAPL_Daily_Return', 'AAPL_Sector', 'AAPL_Market_Cap', 'AAPL_MA_10', 'AAPL_MA_20', 'AAPL_Volatility_10', 'AAPL_Cumulative_Return', 'MA_7', 'MA_30', 'Daily_Return', 'SP500_Close', 'SP500_Daily_Return']


In [ ]:
# -------------------------------
# Comparison plots & summary
# -------------------------------
# AAPL vs S&P closing prices
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=comparison_df['Date'], y=comparison_df[close_col], mode='lines', name=f'{ticker} Close'))
fig4.add_trace(go.Scatter(x=comparison_df['Date'], y=comparison_df['SP500_Close'], mode='lines', name='S&P 500 Close'))
fig4.update_layout(title=f'{ticker} vs S&P 500 Closing Prices', xaxis_title='Date', yaxis_title='Price')
fig4.show()

# AAPL vs S&P daily returns
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=comparison_df['Date'], y=comparison_df['Daily_Return'], mode='lines', name=f'{ticker} Daily Return'))
fig5.add_trace(go.Scatter(x=comparison_df['Date'], y=comparison_df['SP500_Daily_Return'], mode='lines', name='S&P 500 Daily Return'))
fig5.update_layout(title=f'{ticker} vs S&P 500 Daily Returns', xaxis_title='Date', yaxis_title='Daily Return (%)')
fig5.show()

# Correlations & summary table
price_corr = comparison_df[[close_col, 'SP500_Close']].corr().iloc[0,1]
daily_corr = comparison_df[['Daily_Return', 'SP500_Daily_Return']].corr().iloc[0,1]

summary_table = pd.DataFrame({
    'Metric': ['Highest Close', 'Lowest Close', 'Highest Daily Return', 'Lowest Daily Return', 'Price Correlation', 'Daily Return Correlation'],
    'Value': [
        f"{comparison_df[close_col].max():.2f} on {comparison_df.loc[comparison_df[close_col].idxmax(), 'Date']}",
        f"{comparison_df[close_col].min():.2f} on {comparison_df.loc[comparison_df[close_col].idxmin(), 'Date']}",
        f"{comparison_df['Daily_Return'].max():.2f}%",
        f"{comparison_df['Daily_Return'].min():.2f}%",
        f"{price_corr:.2f}",
        f"{daily_corr:.2f}"
    ]
})

print(summary_table)

# Save final CSV
comparison_df.to_csv('AAPL_vs_SP500_analysis.csv', index=False)
print('Saved AAPL_vs_SP500_analysis.csv')


                     Metric                          Value
0             Highest Close  258.02 on 2025-10-03 00:00:00
1              Lowest Close  226.79 on 2025-09-10 00:00:00
2      Highest Daily Return                          4.31%
3       Lowest Daily Return                         -3.23%
4         Price Correlation                           0.82
5  Daily Return Correlation                           0.25
Saved AAPL_vs_SP500_analysis.csv


## Streamlit dashboard

If you want a Streamlit dashboard, save the following script as `dashboard.py` and run `streamlit run dashboard.py` locally or in a cloud VM. It uses the saved `AAPL_vs_SP500_analysis.csv` as input. (We avoid re-downloading data inside Streamlit to keep the notebook deterministic.)

In [ ]:
streamlit_code = '''
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

st.title('AAPL vs S&P 500 Dashboard')

# Load pre-saved CSV
comparison_df = pd.read_csv('AAPL_vs_SP500_analysis.csv')
comparison_df['Date'] = pd.to_datetime(comparison_df['Date'])

fig = go.Figure()
fig.add_trace(go.Scatter(x=comparison_df['Date'], y=comparison_df['AAPL_Close_AAPL'], name='AAPL Close'))
fig.add_trace(go.Scatter(x=comparison_df['Date'], y=comparison_df['SP500_Close'], name='S&P Close'))
st.plotly_chart(fig, use_container_width=True)
'''

print('Streamlit snippet created (save as dashboard.py)')


Streamlit snippet created (save as dashboard.py)
